In [1]:
import os
import pandas as pd
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Folder containing the workbook file
folder_path = "data_files"

# Get all files in the folder
files = os.listdir(folder_path)

# Search for workbook file
workbook_file = None
for file in files:
    if file.endswith(".xlsx"):
        workbook_file = file
        break

# If a workbook file is found
if workbook_file:
    # Path to the workbook file
    file_path = os.path.join(folder_path, workbook_file)

    # Load the workbook
    workbook = load_workbook(file_path, read_only=True)

    # Get the sheet by name
    sheet = workbook['reserve']

    # Load sheet data into a DataFrame
    data = sheet.values
    columns = next(data)  # Assuming the first row contains column names
    df = pd.DataFrame(data, columns=columns)

else:
    print("Workbook file not found in the specified folder.")

In [3]:
# key_fields list
key_fields = ['PYTHON', 'TOTAL_DUT', 'TOTAL_ENG', 'TOTAL_SPA', 'SPORT_TIME',  'EYES', 'RUN', 'CYCLE', 'WRITING']

# Create a new workbook
new_workbook = Workbook()

# Iterate over key_fields
for field in key_fields:
    # Create a new DataFrame with the desired columns
    new_df = df[['WEEK#', 'WEEK_EVENT', field]].copy()

    # Filter out rows where the target column is 0
    new_df = new_df[new_df[field] != 0]

    # Sort the DataFrame by the target column in descending order
    new_df = new_df.sort_values(field, ascending=False)

    # Create a new sheet in the workbook
    new_sheet = new_workbook.create_sheet(title=field)

    # Write the DataFrame to the sheet
    for row in dataframe_to_rows(new_df, index=False, header=True):
        new_sheet.append(row)

# Remove the default sheet created by openpyxl
new_workbook.remove(new_workbook['Sheet'])

# Save the workbook
new_workbook.save('data_files/outcome/summary_book.xlsx')